In [1]:
import xml.etree.ElementTree as ET
import codecs
import json
import re

In [2]:
def get_root(fname):
    tree = ET.parse(fname)
    return tree.getroot()

input_ankara = "ankara_turkey.osm"
root = get_root(input_ankara)

In [3]:
def count_tags(root,tag):
    tag_count = 0
    for i in root.findall(tag):
        tag_count += 1
    return tag_count


def count_users(root):
    user = set()
    for i in root.findall('node') :
        try:
            user.add(i.attrib['user'])
        except:
            pass
    for i in root.findall('way'):
        try:
            user.add(i.attrib['user'])
        except:
            pass
    return  len(user) 
 

In [4]:
counts = {}
for i in ['node', 'way', 'relation' ]:
    counts[i] = count_tags(root,i)
counts['user'] = count_users(root)
print counts

{'node': 638467, 'relation': 220, 'user': 500, 'way': 92056}


In [31]:
def extend_abbrevations(word):
    
    abbrevations = { 'Cd.': 'Caddesi', 'Cad.':'Caddesi', 'Cd':'Caddesi', 'Sk.':'Sokak', 
                    'Mah.':'Mahallesi', 'Blv.':u'Bulvar\u0131', 'Bul.':u'Bulvar\u0131'}
    for i in word.split():
        try:
            cleaned_word = word.replace(i,abbrevations[i]).title()
            print "before cleaning: ", word, "\t after cleaning: ", cleaned_word.title()
        except:
            pass
    return cleaned_word

In [14]:
def correct_postcode(postcode):  #postcodes must start with 06, and must be 5 digits long
    if len(postcode) != 5: # found short postcodes like 0600, 65 in the data
        return None
    elif re.match(r'(06\d\d\d)',postcode)  == None: #check if all digits (i.e. not Esma 3 Sokak) and starts with 06
        print 'postcode', postcode
        return None
    return postcode
    

## Problems ##
1) Abbreviated street, avenue names (e.g. 'Cad.', 'Cd', 'Sk.')

2) Capitalization problems (e.g. 'cad.' , u'kalekap\u0131s\u0131' , 'ANKARA') 

3) Postcodes should consist of exactly 5 digits, and should start with 06.

In [7]:
global k 
k=0

In [30]:
us_ct = 0
uniq_users = set()
def get_value(element, attribute): #inspired from http://goo.gl/1n3xO5, assigns value if the field exists
    try:
        return element.attrib[attribute]
    except:
        return None
    
def shape_element(element):
    global uniq_users
    node = {}
    lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
    problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        node['id'] =  element.attrib['id']
        
        node['created'] = { 'timestamp' : get_value(element,'timestamp'),
                            'version': get_value(element,'version'),
                            'changeset': get_value(element,'changeset'), 
                            'user':get_value(element,'user'), 
                            'uid': get_value(element,'uid')}
        node['visible'] =  get_value(element,'visible')
        
        try:    
            node['pos'] = [float(get_value(element,'lat')), float(get_value(element,'lon'))]
        except:
            pass
        
        try:
            uniq_users.add(get_value(element,'user'))
        except:
            pass
        
        
        for i in element:
        
            if element.tag=="way":
                try:
                    node.setdefault('node_refs',[]).append(i.attrib['ref'])        #http://stackoverflow.com/questions/12905999/python-dict-how-to-create-key-or-append-an-element-to-key
                except:
                    pass
            
            try:
                if "addr" in i.attrib['k']:
                    addr, field = lower_colon.match(i.attrib['k']).group().split(':') 
                    length = len(lower_colon.match(i.attrib['k']).group().split(':'))
                    v = i.attrib['v']

                    if  length <= 2 and addr=='addr' and problemchars.match(v) == None:   
                        if field == 'postcode':
                            v = correct_postcode(v)
                                                    
                        #fix capitalizations, and abbrevations
                        
                        node.setdefault('address',{}).update({ field: extend_abbrevations(v)  }) 
                else:
                     
                    if i.attrib['k'] != 'type': #some entries have <tag k = "type" v=...>
                        node[i.attrib['k']] = i.attrib['v']
                    
            except:
                pass
            
        return node
    else:
  
        return None


In [32]:

def process_map(file_in, pretty = False):
    k = 0
 
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el  = shape_element(element)
            if k > 875100:
                break
            if el:
                k += 1
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    
    return data

In [33]:
clean_ankara = process_map("ankara_turkey.osm")

before cleaning:  Atatürk Blv. 	 after cleaning:  Atatürk Bulvarı
before cleaning:  Vekaletler Cad. 	 after cleaning:  Vekaletler Caddesi
before cleaning:  Tunalı Hilmi Cad. 	 after cleaning:  Tunalı Hilmi Caddesi
before cleaning:  Levent Sk. 	 after cleaning:  Levent Sokak
before cleaning:  Çarşı Cd. 	 after cleaning:  Çarşı Caddesi
before cleaning:  Çarşı Cd. 	 after cleaning:  Çarşı Caddesi
before cleaning:  1608. Cd. 	 after cleaning:  1608. Caddesi
before cleaning:  1608. Cd. 	 after cleaning:  1608. Caddesi
before cleaning:  1608. Cd. 	 after cleaning:  1608. Caddesi
before cleaning:  Yavuzevler Sk. 	 after cleaning:  Yavuzevler Sokak
before cleaning:  Yavuzevler Cd. 	 after cleaning:  Yavuzevler Caddesi
before cleaning:  Ahi Mesud Bul. 	 after cleaning:  Ahi Mesud Bulvarı
before cleaning:  Ahi Mesud Bul. 	 after cleaning:  Ahi Mesud Bulvarı
before cleaning:  Ahi Mesud Bul. 	 after cleaning:  Ahi Mesud Bulvarı
before cleaning:  Nene Hatun Cad. 	 after cleaning:  Nene Hatun Caddes

In [16]:
ct=0
for i in clean_ankara:
    if ct >310:
        break
    
 
    try:
        if 'ABD' in i['name']:
            print i['name'].encode('utf-8')
        # i['address']
        ct += 1
        #if i['address']['city'].islower():
        #    print i['address']['city']
        #    ct +=1 
        #if i['type']=='node' or i['type']=='way':
                 
    except:
        pass
print ct


ABD Büyükelçiliği
311
